In [ ]:
# 기본 라이브러리
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
import numpy as np
import datetime

# 네이버 API를 사용하기 위한 라이브러리
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json


#시각화 라이브러리
import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rc('font', family= 'NanumBarunGothic')
plt.rc('axes', unicode_minus = False)
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

plt.rcParams["figure.figsize"] = (31,10)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = True 

import seaborn as sns
# sns.set(rc={'figure.figsize':(21,10)})
# sns.set_style('ticks')
# sns.set_palette("RdBu")
# sns.set_context('talk')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors


## Raw Data 불러오기

In [ ]:
# 불러올 파일 확인
os.listdir('/content/drive/MyDrive/파이썬 프로젝트/부동산 프로젝트/아파트 실거래가 데이터')

['아파트(매매)__실거래가_2006.csv',
 '아파트(매매)__실거래가_2007.csv',
 '아파트(매매)__실거래가_2008.csv',
 '아파트(매매)__실거래가_2009.csv',
 '아파트(매매)__실거래가_2010.csv',
 '아파트(매매)__실거래가_2011.csv',
 '아파트(매매)__실거래가_2012.csv',
 '아파트(매매)__실거래가_2013.csv',
 '아파트(매매)__실거래가_2014.csv',
 '아파트(매매)__실거래가_2015.csv',
 '아파트(매매)__실거래가_2016.csv',
 '아파트(매매)__실거래가_2017.csv',
 '아파트(매매)__실거래가_2018.csv',
 '아파트(매매)__실거래가_2019.csv',
 '아파트(매매)__실거래가_2020.csv',
 '아파트(매매)__실거래가_2021.csv',
 '아파트(매매)__실거래가_2016.gsheet',
 '아파트(매매)__실거래가_2017.gsheet',
 '아파트(매매)__실거래가_2018.gsheet',
 '아파트(매매)__실거래가_2019.gsheet',
 '아파트(매매)__실거래가_2020.gsheet',
 '아파트(매매)__실거래가_2022.csv']

In [ ]:
# 해당 폴더에 있는 .csv 파일명 확인
path = '/content/drive/MyDrive/파이썬 프로젝트/부동산 프로젝트/아파트 실거래가 데이터/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

In [ ]:
df2 = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i, skiprows = 16, encoding = 'cp949', sep = '|', names = ['거래'])
    df2 = pd.concat([df2,data], ignore_index = True)

df2

,거래
0,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트..."
1,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트..."
2,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트..."
3,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트..."
4,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트..."
...,...
1229220,"서울특별시 중랑구 중화동,""453"",""0453"",""0000"",""청광플러스원"",""84..."
1229221,"서울특별시 중랑구 중화동,""286-22"",""0286"",""0022"",""청원다미소"",""..."
1229222,"서울특별시 중랑구 중화동,""450"",""0450"",""0000"",""한신아파트(103~1..."
1229223,"서울특별시 중랑구 중화동,""274-51"",""0274"",""0051"",""한영(101)""..."


## 전처리

In [ ]:
# 한 컬럼 내 들어간 내용 split을 통해 각각의 내용을 컬럼으로 지정
df2['시군구'] = df2.거래.str.split(',"').str[0]
df2['번지'] = df2.거래.str.split('"').str[1]
df2['본번'] = df2.거래.str.split('","').str[1]
df2['부번'] = df2.거래.str.split('","').str[2]
df2['단지명'] = df2.거래.str.split('","').str[3]
df2['전용면적'] = df2.거래.str.split('","').str[4]
df2['계약연월'] = df2.거래.str.split('","').str[5]
df2['계약일'] = df2.거래.str.split('","').str[6]
df2['거래금액_만원'] = df2.거래.str.split('","').str[7]
df2['층'] = df2.거래.str.split('","').str[8]
df2['건축연도'] = df2.거래.str.split('","').str[9]
df2['도로명'] = df2.거래.str.split('","').str[10]
df2['해제사유발생일'] = df2.거래.str.split('","').str[11]
df2['거래유형'] = df2.거래.str.split('","').str[12]
df2['중개사소재지'] = df2.거래.str.split('","').str[13]
df2

,거래,시군구,번지,본번,부번,단지명,전용면적,계약연월,계약일,거래금액_만원,층,건축연도,도로명,해제사유발생일,거래유형,중개사소재지
0,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200603,10,"59,500",7,1988,언주로 103,,-,"-"""
1,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200603,29,"60,000",6,1988,언주로 103,,-,"-"""
2,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200604,29,"67,000",9,1988,언주로 103,,-,"-"""
3,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,,-,"-"""
4,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,,-,"-"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229220,"서울특별시 중랑구 중화동,""453"",""0453"",""0000"",""청광플러스원"",""84...",서울특별시 중랑구 중화동,453,0453,0000,청광플러스원,84.96,202201,11,"85,500",11,2007,동일로136길 15,,중개거래,"서울 중랑구"""
1229221,"서울특별시 중랑구 중화동,""286-22"",""0286"",""0022"",""청원다미소"",""...",서울특별시 중랑구 중화동,286-22,0286,0022,청원다미소,84.8,202204,7,"76,000",5,2004,동일로136길 10,,중개거래,"서울 중랑구"""
1229222,"서울특별시 중랑구 중화동,""450"",""0450"",""0000"",""한신아파트(103~1...",서울특별시 중랑구 중화동,450,0450,0000,한신아파트(103~109),59.76,202203,27,"73,000",20,1997,동일로 752,,중개거래,"서울 중랑구"""
1229223,"서울특별시 중랑구 중화동,""274-51"",""0274"",""0051"",""한영(101)""...",서울특별시 중랑구 중화동,274-51,0274,0051,한영(101),84.69,202204,9,"49,900",7,2003,동일로144길 74,,중개거래,"서울 중랑구"""


In [ ]:
# 해제사유발생일이 존재하는 거래취소 데이터는 제거
df2 = df2[df2['해제사유발생일'] == '']
df2

,거래,시군구,번지,본번,부번,단지명,전용면적,계약연월,계약일,거래금액_만원,층,건축연도,도로명,해제사유발생일,거래유형,중개사소재지
0,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200603,10,"59,500",7,1988,언주로 103,,-,"-"""
1,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200603,29,"60,000",6,1988,언주로 103,,-,"-"""
2,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200604,29,"67,000",9,1988,언주로 103,,-,"-"""
3,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,,-,"-"""
4,"서울특별시 강남구 개포동,""655-2"",""0655"",""0002"",""개포2차현대아파트...",서울특별시 강남구 개포동,655-2,0655,0002,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,,-,"-"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229220,"서울특별시 중랑구 중화동,""453"",""0453"",""0000"",""청광플러스원"",""84...",서울특별시 중랑구 중화동,453,0453,0000,청광플러스원,84.96,202201,11,"85,500",11,2007,동일로136길 15,,중개거래,"서울 중랑구"""
1229221,"서울특별시 중랑구 중화동,""286-22"",""0286"",""0022"",""청원다미소"",""...",서울특별시 중랑구 중화동,286-22,0286,0022,청원다미소,84.8,202204,7,"76,000",5,2004,동일로136길 10,,중개거래,"서울 중랑구"""
1229222,"서울특별시 중랑구 중화동,""450"",""0450"",""0000"",""한신아파트(103~1...",서울특별시 중랑구 중화동,450,0450,0000,한신아파트(103~109),59.76,202203,27,"73,000",20,1997,동일로 752,,중개거래,"서울 중랑구"""
1229223,"서울특별시 중랑구 중화동,""274-51"",""0274"",""0051"",""한영(101)""...",서울특별시 중랑구 중화동,274-51,0274,0051,한영(101),84.69,202204,9,"49,900",7,2003,동일로144길 74,,중개거래,"서울 중랑구"""


---
---
---

In [ ]:
# 전처리 과정이 오래 걸리는 df2을 원본, df3를 복사본으로 진행
df3 = df2.copy()

In [ ]:
# 불필요한 컬럼 제거
df3 = df3.drop(['거래', '해제사유발생일', '거래유형', '중개사소재지', '번지', '본번', '부번'], axis = 1)
df3

,시군구,단지명,전용면적,계약연월,계약일,거래금액_만원,층,건축연도,도로명
0,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200603,10,"59,500",7,1988,언주로 103
1,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200603,29,"60,000",6,1988,언주로 103
2,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200604,29,"67,000",9,1988,언주로 103
3,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103
4,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103
...,...,...,...,...,...,...,...,...,...
1229220,서울특별시 중랑구 중화동,청광플러스원,84.96,202201,11,"85,500",11,2007,동일로136길 15
1229221,서울특별시 중랑구 중화동,청원다미소,84.8,202204,7,"76,000",5,2004,동일로136길 10
1229222,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,202203,27,"73,000",20,1997,동일로 752
1229223,서울특별시 중랑구 중화동,한영(101),84.69,202204,9,"49,900",7,2003,동일로144길 74


In [ ]:
# 계약연도 컬럼 생성
df3['계약연도'] = df3['계약연월'].str[0:4].astype('int64')
# 계약월 컬럼 생성
df3['계약월'] = df3['계약연월'].str[4:6].astype('int64')
# 구 컬럼 생성
df3['구'] = df3['시군구'].str.split(' ').str[1]
# 동 컬럼 생성
df3['동'] = df3['시군구'].str.split(' ').str[2]

In [ ]:
df3['전용면적'] = df3['전용면적'].astype(float)
df3['거래금액_억원'] = df3['거래금액_만원'].str.replace(',', '').astype('int')
df3['거래금액_만원'] = df3['거래금액_만원'].str.replace(',', '').astype('int')

df3['거래금액_억원'] = (df3['거래금액_만원'] / 10000).round(2)
df3['전용면적_평'] = (df3['전용면적'] / 3.305785).round(0)
df3['평당금액_억원'] = (((df3['거래금액_만원'] * 10000) / df3['전용면적_평']) / 100000000).round(2)

df3

,시군구,단지명,전용면적,계약연월,계약일,거래금액_만원,층,건축연도,도로명,계약연도,계약월,구,동,거래금액_억원,전용면적_평,평당금액_억원
0,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200603,10,59500,7,1988,언주로 103,2006,3,강남구,개포동,5.95,24.0,0.25
1,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200603,29,60000,6,1988,언주로 103,2006,3,강남구,개포동,6.00,24.0,0.25
2,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200604,29,67000,9,1988,언주로 103,2006,4,강남구,개포동,6.70,24.0,0.28
3,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,2006,6,강남구,개포동,6.00,24.0,0.25
4,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,2006,10,강남구,개포동,7.22,24.0,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229220,서울특별시 중랑구 중화동,청광플러스원,84.96,202201,11,85500,11,2007,동일로136길 15,2022,1,중랑구,중화동,8.55,26.0,0.33
1229221,서울특별시 중랑구 중화동,청원다미소,84.80,202204,7,76000,5,2004,동일로136길 10,2022,4,중랑구,중화동,7.60,26.0,0.29
1229222,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,202203,27,73000,20,1997,동일로 752,2022,3,중랑구,중화동,7.30,18.0,0.41
1229223,서울특별시 중랑구 중화동,한영(101),84.69,202204,9,49900,7,2003,동일로144길 74,2022,4,중랑구,중화동,4.99,26.0,0.19


In [ ]:
df3 = df3.reindex(columns = ['시군구', '구','동', '도로명', '단지명', '건축연도', '계약연월', '계약연도', '계약월', '계약일', '층', '거래금액_억원', '전용면적_평', '평당금액_억원'])
df3

,시군구,구,동,도로명,단지명,건축연도,계약연월,계약연도,계약월,계약일,층,거래금액_억원,전용면적_평,평당금액_억원
0,서울특별시 강남구 개포동,강남구,개포동,언주로 103,개포2차현대아파트(220),1988,200603,2006,3,10,7,5.95,24.0,0.25
1,서울특별시 강남구 개포동,강남구,개포동,언주로 103,개포2차현대아파트(220),1988,200603,2006,3,29,6,6.00,24.0,0.25
2,서울특별시 강남구 개포동,강남구,개포동,언주로 103,개포2차현대아파트(220),1988,200604,2006,4,29,9,6.70,24.0,0.28
3,서울특별시 강남구 개포동,강남구,개포동,언주로 103,개포2차현대아파트(220),1988,200606,2006,6,1,4,6.00,24.0,0.25
4,서울특별시 강남구 개포동,강남구,개포동,언주로 103,개포2차현대아파트(220),1988,200610,2006,10,20,5,7.22,24.0,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229220,서울특별시 중랑구 중화동,중랑구,중화동,동일로136길 15,청광플러스원,2007,202201,2022,1,11,11,8.55,26.0,0.33
1229221,서울특별시 중랑구 중화동,중랑구,중화동,동일로136길 10,청원다미소,2004,202204,2022,4,7,5,7.60,26.0,0.29
1229222,서울특별시 중랑구 중화동,중랑구,중화동,동일로 752,한신아파트(103~109),1997,202203,2022,3,27,20,7.30,18.0,0.41
1229223,서울특별시 중랑구 중화동,중랑구,중화동,동일로144길 74,한영(101),2003,202204,2022,4,9,7,4.99,26.0,0.19


In [ ]:
df3 = df3.sort_values(by = ['계약연월','전용면적_평'], ascending = True)
df3 = df3.reset_index(drop=True)
df3

,시군구,구,동,도로명,단지명,건축연도,계약연월,계약연도,계약월,계약일,층,거래금액_억원,전용면적_평,평당금액_억원
0,서울특별시 강남구 역삼동,강남구,역삼동,언주로90길 6,우림루미아트1,2003,200601,2006,1,3,8,1.05,7.0,0.15
1,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,6,5,2.53,8.0,0.32
2,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,6,3,2.60,8.0,0.32
3,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,17,1,2.64,8.0,0.33
4,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,18,4,2.70,8.0,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224693,서울특별시 서초구 잠원동,서초구,잠원동,신반포로23길 80,오페라하우스,1997,202204,2022,4,26,10,29.35,70.0,0.42
1224694,서울특별시 서초구 반포동,서초구,반포동,동광로27길 84,아델스빌1,2007,202204,2022,4,8,6,43.00,73.0,0.59
1224695,서울특별시 성동구 성수동1가,성동구,성수동1가,서울숲2길 32-14,갤러리아포레,2011,202204,2022,4,28,24,78.50,73.0,1.08
1224696,서울특별시 서초구 반포동,서초구,반포동,동광로27길 67,선데일,2006,202204,2022,4,4,3,39.80,74.0,0.54


---

In [ ]:
df3['계약날짜'] = df3['계약연도'].map(str) + '-' + df3['계약월'].map(str) + '-' + df3['계약일'].map(str)
df3['계약날짜'] = pd.to_datetime(df3['계약날짜'])
df3

,시군구,구,동,도로명,단지명,건축연도,계약연월,계약연도,계약월,계약일,층,거래금액_억원,전용면적_평,평당금액_억원,계약날짜
0,서울특별시 강남구 역삼동,강남구,역삼동,언주로90길 6,우림루미아트1,2003,200601,2006,1,3,8,1.05,7.0,0.15,2006-01-03
1,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,6,5,2.53,8.0,0.32,2006-01-06
2,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,6,3,2.60,8.0,0.32,2006-01-06
3,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,17,1,2.64,8.0,0.33,2006-01-17
4,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,18,4,2.70,8.0,0.34,2006-01-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224693,서울특별시 서초구 잠원동,서초구,잠원동,신반포로23길 80,오페라하우스,1997,202204,2022,4,26,10,29.35,70.0,0.42,2022-04-26
1224694,서울특별시 서초구 반포동,서초구,반포동,동광로27길 84,아델스빌1,2007,202204,2022,4,8,6,43.00,73.0,0.59,2022-04-08
1224695,서울특별시 성동구 성수동1가,성동구,성수동1가,서울숲2길 32-14,갤러리아포레,2011,202204,2022,4,28,24,78.50,73.0,1.08,2022-04-28
1224696,서울특별시 서초구 반포동,서초구,반포동,동광로27길 67,선데일,2006,202204,2022,4,4,3,39.80,74.0,0.54,2022-04-04


In [ ]:
# 데이터가 언제부터 언제까지 있는지 확인 
print('데이터 기간')
print('처음 :', df3['계약날짜'].min())
print('마지막 : ', df3['계약날짜'].max())

데이터 기간
처음 : 2006-01-01 00:00:00
마지막 :  2022-04-30 00:00:00


In [ ]:
# 데이터의 총 개수확인
print('데이터 총 개수 : ', len(df3.index))

데이터 총 개수 :  1224698


In [ ]:
# 위도, 경도 컬럼을 제외한 전처리가 모두 완료된 df3 파일에서 df를 복사본으로 진행
df = df3.copy()

In [ ]:
# 기존의 도로명 컬럼은 122만개의 중복이 포함되어있어, 빠른 지오코딩(주소 -> 위도,경도)을 위해 중복을 제외한 약 9000개의 값으로 찾기 진행
도로명 = pd.DataFrame(df['도로명'].drop_duplicates(keep= 'first'))
도로명['도로명'].values

array(['언주로90길 6', '선릉로 8', '테헤란로64길 9', ..., '서초대로58길 18', '양천로24길 13',
       '대흥로24길 24'], dtype=object)

In [ ]:
# naver api
client_id = 'li8opwpdmx'    # 본인이 할당받은 ID 입력
client_pw = 'bNzrhQf2b2UHobEIZzZYg29sSsH8qVBRspcOYJvz'    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 주소 목록 파일 (.xlsx)
data = 도로명



# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in data['도로명']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])


np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"도로명": data['도로명'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
'result' not exist!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!

In [ ]:
# 중복된 도로명은 제외하여 지오코딩(도로명->위도,경도)를 진행하였기 때문에 전체의 데이터 프레임의 도로명 주소에 Join을 진행
df = pd.merge(df, pd_geo_coordi, how = 'left', on = '도로명')

# Join이 완료된 데이터 프레임을 CSV 파일로 저장 
df.to_csv('/content/drive/MyDrive/파이썬 프로젝트/부동산 프로젝트/Geocoding 완료 파일.csv')
df

,시군구,구,동,도로명,단지명,건축연도,계약연월,계약연도,계약월,계약일,...,계약날짜,구역별,평당_금액대,평형대,브랜드명,브랜드여부,위도_x,경도_x,위도_y,경도_y
0,서울특별시 강남구 역삼동,강남구,역삼동,언주로90길 6,우림루미아트1,2003,200601,2006,1,3,...,2006-01-03,강남,0 - 2000 사이,18평 이하,이외브랜드,0,37.5021957,127.0437307,37.5021957,127.0437307
1,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,6,...,2006-01-06,강남,2000 - 4000 사이,18평 이하,이외브랜드,0,37.4810560,127.0641830,37.4810560,127.0641830
2,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,6,...,2006-01-06,강남,2000 - 4000 사이,18평 이하,이외브랜드,0,37.4810560,127.0641830,37.4810560,127.0641830
3,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,17,...,2006-01-17,강남,2000 - 4000 사이,18평 이하,이외브랜드,0,37.4810560,127.0641830,37.4810560,127.0641830
4,서울특별시 강남구 개포동,강남구,개포동,선릉로 8,주공2,1982,200601,2006,1,18,...,2006-01-18,강남,2000 - 4000 사이,18평 이하,이외브랜드,0,37.4810560,127.0641830,37.4810560,127.0641830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224693,서울특별시 서초구 잠원동,서초구,잠원동,신반포로23길 80,오페라하우스,1997,202204,2022,4,26,...,2022-04-26,강남,4000 - 6000 사이,40평 초과,이외브랜드,0,37.5122461,127.0058987,37.5122461,127.0058987
1224694,서울특별시 서초구 반포동,서초구,반포동,동광로27길 84,아델스빌1,2007,202204,2022,4,8,...,2022-04-08,강남,4000 - 6000 사이,40평 초과,이외브랜드,0,37.4969019,126.9937115,37.4969019,126.9937115
1224695,서울특별시 성동구 성수동1가,성동구,성수동1가,서울숲2길 32-14,갤러리아포레,2011,202204,2022,4,28,...,2022-04-28,동서울,1 - 1.2억 사이,40평 초과,이외브랜드,0,37.5457261,127.0423333,37.5457261,127.0423333
1224696,서울특별시 서초구 반포동,서초구,반포동,동광로27길 67,선데일,2006,202204,2022,4,4,...,2022-04-04,강남,4000 - 6000 사이,40평 초과,이외브랜드,0,37.4962587,126.9934932,37.4962587,126.9934932
